# vss

This was a pretty fun challenge! But since this notebook is a terrible mess I'm not at this point planning to turn this into a properly descriptive write-up. So it's mainly just for completeness, and you can get a better write-up from either [Mystiz](https://mystiz.hk/posts/2022/2022-09-05-balsn/) or [tl2cents](https://tl2cents.github.io/2022/09/05/2022-BalsnCTF-Crypto-Writeup/). Especially since we all solved it basically the same way. Maybe if there's demand I can update in the future though.

In [1]:
from pwn import *
from tqdm import tqdm
from sage.all import *
from sage.rings.factorint import factor_trial_division

from Crypto.Util.number import *
from Crypto.Cipher import AES
from hashlib import sha256

In [2]:
flag = '7ab643d7e7ca00e4990500e704d7d01284682f2d28cd8c7c98e73b209c23992209c248aeecf2bd0d64d37928aa7e08a4'
# parsed separately from a text file: format is (p, a, b, c, q, g, y)
# where p is the getShare prime and q is the commit prime
alldata=[(...)]
qs = [row[4] for row in alldata]

In [3]:
def bigfactor(n, limit=2**24):
    return product(a**b for a,b in factor_trial_division(n,limit)[:-1])

bfs = [bigfactor(q-1) for q in tqdm(qs)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [00:11<00:00, 13.91it/s]


In [4]:
# take the 20 best indices
size = 20
best_indices = [a for a,b in sorted(enumerate(bfs), key=lambda p:-p[1])[:size]]
print(best_indices)

[78, 73, 109, 79, 98, 104, 59, 11, 159, 4, 156, 19, 100, 22, 90, 5, 42, 126, 62, 21]


In [5]:
# determine the actual multiplicative_order(), which is a factor of d
def real_mult_order(dn, d):
    for p,e in factor(d):
        while d % p == 0 and dn**(d//p)==1:
            d //= p
    return d

# e is the discrete log (mod d)
def get_de_value(q,g,y,d):
    assert (q-1)%d==0
    up = GF(q)(y)**((q-1)//d)
    dn = GF(q)(g)**((q-1)//d)
    d = real_mult_order(dn, d)
    e = discrete_log(up, dn, d)
    return (d, e)

In [6]:
smalldata = []
for i in best_indices:
    (p,a,b,c,q,g,y),d=alldata[i],bfs[i]
    de=get_de_value(q,g,y,d)
    print(i,de[0])
    smalldata.append((p,a,b,c,q,g,y,de[0],de[1]))

78 248309686614075120878932328
73 9137564747405276204952
109 364263416266145239088
79 37130102293792592126
98 20463616860227291262
104 9352991679126955223
59 3345225206507123959
11 651651974345693302
159 154602621468082628
4 10668076785449562
156 35849067897126632
19 3502496847651914
100 41299829574672018
22 81640892671444588
90 26473617232149532
5 4104574460227200
42 381850204282321
126 83657288443651
62 266693165201430
21 407675314955146


In [7]:
#smalldata

In [8]:
lcm([row[7] for row in smalldata]).nbits()

1070

In [9]:
mat = matrix(2*size+2, 3*size+2)
for i in range(2*size+2):
    mat[i,i] = 1
    
for i in range(size):
    mat[0,2*size+2+i] = smalldata[i][2] # b
    mat[1,2*size+2+i] = smalldata[i][3] # c
    mat[2+i,2*size+2+i] = -smalldata[i][0] # p
    mat[size+2+i, 2*size+2+i] = -smalldata[i][7] # d
    
print(mat[-1])

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -407675314955146)


In [10]:
target1 = [(0, 2**512)]*2 + [(0,2**514)]*size
target2 = [(0, row[0]//row[7]) for row in smalldata]
target3 = [row[8] - row[1] for row in smalldata]
target3 = [(t,t) for t in target3]
target = target1 + target2 + target3

In [11]:
import cvp
lb = [a for a,b in target]
ub = [b for a,b in target]
soln = cvp.solve(mat,lb,ub)

key = long_to_bytes(soln[0][0]//soln[1][0]) + long_to_bytes(soln[0][1]//soln[1][1])
real_key = sha256(key).digest()[:16]
cipher = AES.new(real_key, AES.MODE_ECB)
print(cipher.decrypt(bytes.fromhex(flag)))

b'BALSN{commitments_leak_too_much_QwQ}\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c'
